# Исследование надежности заемщиков.

Цель исследования оценить влияет ли семейное положение и количество детей клиента на факт погашения кредита в срок. Входные данные — статистика о платёжеспособности клиентов.

В исходных данных содержится следующая информация:
- children — количество детей в семье
- days_employed — общий трудовой стаж в днях
- dob_years — возраст клиента в годах
- education — уровень образования клиента
- education_id — идентификатор уровня образования
- family_status — семейное положение
- family_status_id — идентификатор семейного положения
- gender — пол клиента
- income_type — тип занятости
- debt — имел ли задолженность по возврату кредитов
- total_income — ежемесячный доход
- purpose — цель получения кредита

### Шаг 1. Обзор данных

In [1]:
import pandas as pd

In [2]:
data=pd.read_csv('/datasets/data.csv')

In [3]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21525 entries, 0 to 21524
Data columns (total 12 columns):
children            21525 non-null int64
days_employed       19351 non-null float64
dob_years           21525 non-null int64
education           21525 non-null object
education_id        21525 non-null int64
family_status       21525 non-null object
family_status_id    21525 non-null int64
gender              21525 non-null object
income_type         21525 non-null object
debt                21525 non-null int64
total_income        19351 non-null float64
purpose             21525 non-null object
dtypes: float64(2), int64(5), object(5)
memory usage: 2.0+ MB


In [4]:
data.head()

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
0,1,-8437.673028,42,высшее,0,женат / замужем,0,F,сотрудник,0,253875.639453,покупка жилья
1,1,-4024.803754,36,среднее,1,женат / замужем,0,F,сотрудник,0,112080.014102,приобретение автомобиля
2,0,-5623.422610,33,Среднее,1,женат / замужем,0,M,сотрудник,0,145885.952297,покупка жилья
3,3,-4124.747207,32,среднее,1,женат / замужем,0,M,сотрудник,0,267628.550329,дополнительное образование
4,0,340266.072047,53,среднее,1,гражданский брак,1,F,пенсионер,0,158616.077870,сыграть свадьбу


### Шаг 2.1 Заполнение пропусков

В исходных данных есть пропущенные значения в столбцах days_employed и total_income. Количество пустых ячеек совпадает, возможно заемщики не работают и не имеют доход. 

In [5]:
data['days_employed'].isna().sum()/data['children'].count()

0.10099883855981417

Доля пропущенных значений составляет 10%. Для дальнейшего анализа пустные ячейки заполнены медианным значением по столбцу. В таблице есть сильно выделяющиеся значения, поэтому расчет среднего показателя может быть сильно завышен. Медианное значение лучше отражает средние показатели. 

In [6]:
days_med = abs(data['days_employed']).median() #расчет медианы для количества отработанных дней

In [7]:
data['median'] = data.groupby('income_type')['total_income'].transform('median') #расчет медианы для суммы дохода

In [8]:
# заполнение пустых ячеек медианным значением
data['days_employed'] = data['days_employed'].fillna(days_med)
data['total_income'] = data['total_income'].fillna(data['median'])

### Шаг 2.2 Проверка данных на аномалии и исправления.

In [9]:
data.sort_values(by='days_employed', ascending=False).head()

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose,median
6954,0,401755.400475,56,среднее,1,вдовец / вдова,2,F,пенсионер,0,176278.441171,ремонт жилью,118514.486412
10006,0,401715.811749,69,высшее,0,Не женат / не замужем,4,F,пенсионер,0,57390.256908,получение образования,118514.486412
7664,1,401675.093434,61,среднее,1,женат / замужем,0,F,пенсионер,0,126214.519212,операции с жильем,118514.486412
2156,0,401674.466633,60,среднее,1,женат / замужем,0,M,пенсионер,0,325395.724541,автомобили,118514.486412
7794,0,401663.850046,61,среднее,1,гражданский брак,1,F,пенсионер,0,48286.441362,свадьба,118514.486412


1. В столбце с количеством отработанных дней есть отрицательные значения и дробная часть. Также есть сомнения в том что данные соответствуют действительности. При переводе максимального значения в количество лет выдает значение 1000. Возможно время указано в часах. Данная информация не влияет на анализ, поэтому можно оставить без изменений.
2. В колонке количество детей есть отрицательные значения. Также много строк со значением 20. Скорее всего заполнено по ошибке. В отрицательных значениях был изменен знак. Строки со значением 20 заменены на значение 2, т.к. предполагаю что 0 - лишний символ в ячейке.

In [10]:
data['children'] = abs(data['children']) # изменение отрицательного знака на положительный
data.loc[data['children']==20, 'children']=2 #  замена значения 20 на 2
    

3. В колонке gender есть значение XNA. Поскольку с таким значением только 1 строка, ее можно удалить, она не повлияет на результат анализа.

In [11]:
data = data[data['gender']!='XNA'] # из датасета удалена строка с некорректным полом

### Шаг 2.3. Изменение типов данных.

In [12]:
data['total_income'] = data['total_income'].astype('int')

### Шаг 2.4. Удаление дубликатов.

Поиск явных дубликатов проведен методом duplicated().sum(). 

In [13]:
data.duplicated().sum()

54

In [14]:
data = data.drop_duplicates() # удаление дубликатов строк

In [15]:
data['education'].unique() # просмотр уникальных значений

array(['высшее', 'среднее', 'Среднее', 'СРЕДНЕЕ', 'ВЫСШЕЕ',
       'неоконченное высшее', 'начальное', 'Высшее',
       'НЕОКОНЧЕННОЕ ВЫСШЕЕ', 'Неоконченное высшее', 'НАЧАЛЬНОЕ',
       'Начальное', 'Ученая степень', 'УЧЕНАЯ СТЕПЕНЬ', 'ученая степень'],
      dtype=object)

In [16]:
data['education'] = data['education'].str.lower() # приведение данных к единому формату

In [17]:
data['education'].unique()

array(['высшее', 'среднее', 'неоконченное высшее', 'начальное',
       'ученая степень'], dtype=object)

Данные об образовании записаны по-разному, с использованием верхнего и нижнего регистра. Для устранения дубликатов, достаточно привести все значения к нижнему регистру.
Подобные ошибки возникают когда нет установленного требования к заполнению типичных данных. Лучше создавать уникальные списки и пользоваться значением из списка для заполнения данных.

### Шаг 2.5. Формирование дополнительных датафреймов словарей, декомпозиция исходного датафрейма.

In [18]:
edu = data[['education_id', 'education']].drop_duplicates().reset_index(drop=True) #сформирован справочник образования
display(edu)

,education_id,education
0,0,высшее
1,1,среднее
2,2,неоконченное высшее
3,3,начальное
4,4,ученая степень


In [19]:
family = data[['family_status_id', 'family_status']].drop_duplicates().reset_index(drop=True) #сформирован справочник семейного положения
display(family)

,family_status_id,family_status
0,0,женат / замужем
1,1,гражданский брак
2,2,вдовец / вдова
3,3,в разводе
4,4,Не женат / не замужем


In [20]:
data = data.drop(columns= 'education', axis=1)
data = data.drop(columns= 'family_status', axis=1) 
# удалены столбцы с образованием и семейным положением

In [21]:
data.head(10)

,children,days_employed,dob_years,education_id,family_status_id,gender,income_type,debt,total_income,purpose,median
0,1,-8437.673028,42,0,0,F,сотрудник,0,253875,покупка жилья,142594.396847
1,1,-4024.803754,36,1,0,F,сотрудник,0,112080,приобретение автомобиля,142594.396847
2,0,-5623.422610,33,1,0,M,сотрудник,0,145885,покупка жилья,142594.396847
3,3,-4124.747207,32,1,0,M,сотрудник,0,267628,дополнительное образование,142594.396847
4,0,340266.072047,53,1,1,F,пенсионер,0,158616,сыграть свадьбу,118514.486412
5,0,-926.185831,27,0,1,M,компаньон,0,255763,покупка жилья,172357.950966
6,0,-2879.202052,43,0,0,F,компаньон,0,240525,операции с жильем,172357.950966
7,0,-152.779569,50,1,0,M,сотрудник,0,135823,образование,142594.396847
8,2,-6929.865299,35,0,1,F,сотрудник,0,95856,на проведение свадьбы,142594.396847
9,0,-2188.756445,41,1,0,M,сотрудник,0,144425,покупка жилья для семьи,142594.396847


### Шаг 2.6. Категоризация дохода.

In [22]:
def income_rate(income):
    if income <= 30000:
        return 'E'
    if income <= 50000:
        return 'D'
    if income <= 200000:
        return 'C'
    if income <= 1000000:
        return 'B'
    return 'A'
#создана функция для определния категории дохода

In [23]:
data['total_income_category'] = data['total_income'].apply(income_rate)

In [24]:
data.head()

,children,days_employed,dob_years,education_id,family_status_id,gender,income_type,debt,total_income,purpose,median,total_income_category
0,1,-8437.673028,42,0,0,F,сотрудник,0,253875,покупка жилья,142594.396847,B
1,1,-4024.803754,36,1,0,F,сотрудник,0,112080,приобретение автомобиля,142594.396847,C
2,0,-5623.422610,33,1,0,M,сотрудник,0,145885,покупка жилья,142594.396847,C
3,3,-4124.747207,32,1,0,M,сотрудник,0,267628,дополнительное образование,142594.396847,B
4,0,340266.072047,53,1,1,F,пенсионер,0,158616,сыграть свадьбу,118514.486412,C


### Шаг 2.7. Категоризация целей кредита.

In [25]:
data['purpose'].unique()

array(['покупка жилья', 'приобретение автомобиля',
       'дополнительное образование', 'сыграть свадьбу',
       'операции с жильем', 'образование', 'на проведение свадьбы',
       'покупка жилья для семьи', 'покупка недвижимости',
       'покупка коммерческой недвижимости', 'покупка жилой недвижимости',
       'строительство собственной недвижимости', 'недвижимость',
       'строительство недвижимости', 'на покупку подержанного автомобиля',
       'на покупку своего автомобиля',
       'операции с коммерческой недвижимостью',
       'строительство жилой недвижимости', 'жилье',
       'операции со своей недвижимостью', 'автомобили',
       'заняться образованием', 'сделка с подержанным автомобилем',
       'получение образования', 'автомобиль', 'свадьба',
       'получение дополнительного образования', 'покупка своего жилья',
       'операции с недвижимостью', 'получение высшего образования',
       'свой автомобиль', 'сделка с автомобилем',
       'профильное образование', 'высшее об

In [26]:
def purpose_category(purpose):
    if 'автомобил' in purpose:
        return 'операции с автомобилем'
    elif 'жиль' in purpose or 'недвижим' in purpose:
        return 'операции с недвижимостью'
    elif 'свадьб' in purpose:
        return 'проведение свадьбы'
    elif 'образовани' in purpose:
        return 'получение образования'
    return 'прочее'
# функция для определения категори цели займа

In [27]:
data['purpose_category'] = data['purpose'].apply(purpose_category)

In [28]:
data.head()

,children,days_employed,dob_years,education_id,family_status_id,gender,income_type,debt,total_income,purpose,median,total_income_category,purpose_category
0,1,-8437.673028,42,0,0,F,сотрудник,0,253875,покупка жилья,142594.396847,B,операции с недвижимостью
1,1,-4024.803754,36,1,0,F,сотрудник,0,112080,приобретение автомобиля,142594.396847,C,операции с автомобилем
2,0,-5623.422610,33,1,0,M,сотрудник,0,145885,покупка жилья,142594.396847,C,операции с недвижимостью
3,3,-4124.747207,32,1,0,M,сотрудник,0,267628,дополнительное образование,142594.396847,B,получение образования
4,0,340266.072047,53,1,1,F,пенсионер,0,158616,сыграть свадьбу,118514.486412,C,проведение свадьбы


### Ответы на вопросы.

Для анализа составила сводную таблицу и расчитала долю имевших задолженность заемиков в общей доле заемщиков в категории.

##### Вопрос 1: Есть ли зависимость между количеством детей и возвратом кредита в срок?

In [29]:
data_children = data.pivot_table(index = ['children'], columns='debt', values='income_type', aggfunc='count').fillna(0)
data_children['share'] = data_children[1]/(data_children[0]+data_children[1])*100
display(data_children)

debt,0,1,share
children,,,
0,13043.0,1063.0,7.535800
1,4411.0,445.0,9.163921
2,1926.0,202.0,9.492481
3,303.0,27.0,8.181818
4,37.0,4.0,9.756098
5,9.0,0.0,0.000000


##### Вывод 1:

Заемщики с 5 детьми не имели просроченной задолженности по кредитам, но и количество таких заемщиков очень низкое. Для оценки надежности заемщиков мало данных.
Заемщики без детей имеют самую низкую долю задолженности по кредитам, при этом в этой категории больше всего заявок.
Заемщики с 1 - 4 детьми имеют примерно равную долю задолженности по кредитам.
Можно сделать вывод что количество детей не влияет на возврат кредита. Но наличие или отсутствие детей влияет. Заемщики без детей являются самыми надежными.

##### Вопрос 2: Есть ли зависимость между семейным положением и возвратом кредита в срок?

In [30]:
data_f_status = data.pivot_table(index = ['family_status_id'], columns='debt', values='income_type', aggfunc='count')
data_f_status['share'] = data_f_status[1]/(data_f_status[0]+data_f_status[1])*100
display(data_f_status.merge(family, on='family_status_id', how='left'))

,family_status_id,0,1,share,family_status
0,0,11413,931,7.542126,женат / замужем
1,1,3774,388,9.322441,гражданский брак
2,2,896,63,6.569343,вдовец / вдова
3,3,1110,85,7.112971,в разводе
4,4,2536,274,9.750890,Не женат / не замужем


##### Вывод 2:

Самая низкая доля просроченной задолженности по кредитам у 2 категории заемщиков (вдовцы) - 6,6%
Доля женатых и в разводе заемщиков имевших просроченные кредиты примерно одинаковая 7,1-7,5%
Самая большая доля заемщиков имевших просроченную задолженность по кредитам среди неженатых и живущих в гражданском браке. - больше 9%. Соответственно это самые ненадежные заемщики. Думаю это связано с тем что неженатый человек в случае потери дохода не может оплачивать кредит, в то время как женатый может переложить оплату кредита на супруга на время отсутствия дохода.

##### Вопрос 3: Есть ли зависимость между уровнем дохода и возвратом кредита в срок?

In [31]:
data_income = data.pivot_table(index = ['total_income_category'], columns='debt', values='income_type', aggfunc='count')
data_income['share'] = data_income[1]/(data_income[0]+data_income[1])*100
display(data_income)

debt,0,1,share
total_income_category,,,
A,23,2,8.000000
B,4685,356,7.062091
C,14672,1360,8.483034
D,329,21,6.000000
E,20,2,9.090909


##### Вывод 3:

Заемщики с самым низким уровнем дохода имеют самую большую долю просроченной задолженности по кредитам. Самая низкая доля у заемщиков с уровнем дохода от 30 до 50 тыс руб. 
Доля просроченной задолженности у заемщиков с более высоким уровнем дохода выше. Соответвенно уровень дохода влияет на возврат кредитов, но с повышением уровня дохода надежность заемщиков снижается.

##### Вопрос 4: Как разные цели кредита влияют на его возврат в срок?

In [32]:
data_purpose = data.pivot_table(index = ['purpose_category'], columns='debt', values='income_type', aggfunc='count')
data_purpose['share'] = data_purpose[1]/(data_purpose[0]+data_purpose[1])*100
display(data_purpose)

debt,0,1,share
purpose_category,,,
операции с автомобилем,3905,403,9.354689
операции с недвижимостью,10031,782,7.232036
получение образования,3644,370,9.217738
проведение свадьбы,2149,186,7.965739


##### Вывод 4:

Кредиты на получение образования и покупку автомобиля явяются самыми ненадежными. Доля просроченных кредитов более 9%
Кредиты на проведение свадьбы и покупку недвижимости явяются более надежными. Доля просроченных кредитов 7,2-8 %.

## Общий вывод:

Проведен анализ влияния семейного положения и количество детей клиента на факт погашения кредита в срок.

В исходных данных было 10 % пропущенных значений уровня дохода, которые могли повлиять на результат анализа. Для снижения влияния уровень дохода был приведен к медианному значению.
Также в столбце с количеством детей заемщика были допущены ошибки, например отрицательные значения или слишком большое количество. Отрицательные значения изменены на положительные, а количество детей 20 заменено на 2.
Исходные данные были очищены от дублируюих записей.

Для анализа заемщики были разделены на категории по количеству детей, семейному положению, уровню дохода и цели получения кредита.

Меньше всего доля просроченных кредитов среди заемщиков:
- не имеющих детей;
- вдовцов, женатых или в разводе;
- с уровнем дохода 30-50 тыс.;
- с целью покупки недвижимости.

Больше всего доля просроченных платежей по кредиту среди заемщиков:
- имеющих детей;
- не женатых, либо живущих в гражданском браке;
- с уровнем дохода 50-200 тыс и более 1 млн.;
- с целью приобретения автомобиля.


In [33]:
import datetime as dt
dt.datetime.today().strftime("%d.%m.%Y %H:%M")

'23.02.2022 18:17'